In [ ]:
from typing import List, Sequence

from langchain_groq.chat_models import ChatGroq
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import END, MessageGraph

from dotenv import load_dotenv, find_dotenv

# Load environment variables from.env file
load_dotenv(find_dotenv(), override=True)

In [ ]:
llm = ChatGroq(model="llama-3.1-70b-versatile",
                        stop_sequences="[end]")

In [ ]:
# creating a chat prompt template
generation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            '''You're a Twitter expert tasked with crafting exceptional tweets. 
            Your mission is to create the most engaging and impactful tweet possible based on the user's request.
            If the user provides feedback, you'll refine and enhance your previous attempts to maximize engagement and deliver outstanding results.''',
        ),
        MessagesPlaceholder(variable_name='messages'),
    ]
)

generation_chain  = generation_prompt | llm

In [ ]:
generation_chain

In [ ]:
tweet = ''
request = HumanMessage(
    content='Olymics 2024'
)

for chunk in generation_chain.stream({'messages': [request]}): 
    print(chunk.content, end='')
    tweet += chunk.content

## Reflection

In [ ]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            '''You're a highly influential Twitter personality renowned for crafting captivating content and sharing astute observations.
            Review and critique the user’s tweet.
            To effectively refine the work, you should provide constructive feedback that targets key areas for improvement, such as increasing its depth, refining its style, and amplifying its overall impact.
            You can make the tweet more compelling and engaging for their audience by offering specific suggestions.'''
        ),
        MessagesPlaceholder(variable_name='messages'),
    ]
)

reflection_chain  = reflection_prompt | llm

In [ ]:
reflection_chain

In [ ]:
tweet

In [ ]:
reflection = ''

for chunk in reflection_chain.stream(
    {'messages': [request, HumanMessage(content=tweet)]}
):
    print(chunk.content, end='')
    reflection += chunk.content

## Reflectin using MessageGraph

In [ ]:
generation_chain

In [ ]:
tweet = ''
request = HumanMessage(
    content='Olymics 2024'
)
generation_chain.invoke({'messages': [request]})

In [ ]:
[request]

In [ ]:
def generation_node(state: Sequence[BaseMessage]) -> List[BaseMessage]:
    return generation_chain.invoke([state])

def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    role_map = {'ai': HumanMessage, 'human': AIMessage}

    updated_messages = [messages[0]] + [role_map[msg.type](content=msg.content) for msg in messages[1:]]
    response = reflection_chain.invoke({'messages': updated_messages})
    return HumanMessage(response.content)

In [ ]:
graph_builder = MessageGraph()

graph_builder.add_node('generate', generation_node)
graph_builder.add_node('reflect', reflection_node)
graph_builder.set_entry_point('generate')
# graph_builder.set_finish_point('generate')

MAX_ITERATIONS = 5
def should_continue(state: List[BaseMessage]):
    if len(state) > MAX_ITERATIONS:
        return END
    return 'reflect'

graph_builder.add_conditional_edges('generate', should_continue)
graph_builder.add_edge('reflect', 'generate')

message_graph = graph_builder.compile()